In [4]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

ModuleNotFoundError: No module named 'psycopg2'

In [5]:
# 1 Add the function that takes in  arguments for each combined data file;


# Read in the files as Pandas DataFrames.

postsecdata_to_load = "postsecondary_enrollment_combined_1718_1819_1920.csv"
budgetrollup_to_load = "combined_rollup_budget_data_1718_1819.csv"
costdata_to_load = "comp_cost_combined_1718_1819_1920_2021.csv"
revenuedata_to_load = "comp_rev_combined_1718_1819_1920_2021.csv"
reportcarddata_to_load = "district_reportcard_combined_1718_1819_2021.csv"
masterlist_to_load = "Master_Schools_list.csv"

post_sec_df = pd.read_csv(postsecdata_to_load)
budget_df = pd.read_csv(budgetrollup_to_load)
cost_df = pd.read_csv(costdata_to_load)
revenue_df = pd.read_csv(revenuedata_to_load)
reportcard_df = pd.read_csv(reportcarddata_to_load)
masterlist_df = pd.read_csv(masterlist_to_load)

In [7]:
reportcard_df.count()

School_Year                                          1264
District_ID                                          1264
School_Code                                           885
Cooperative_Educational_Service_Agency               1264
District_Name                                        1264
Overall_Accountability_Score                         1261
Overall_Accountability_Rating                        1261
Lowest_Grade_in_the_District                         1264
Highest_Grade_in_the_District                        1264
School_Type                                           133
District_Enrollment                                  1264
Percent_American_Indian_or_Alaskan_Native            1264
Percent_Asian                                        1264
Percent_Black_or_African_American                    1264
Percent_Hispanic_Latino                              1264
Percent_Native_Hawaiian_or_Other_Pacific_Islander    1264
Percent_White                                        1264
Percent_Two_or

In [8]:
#check for missing values
reportcard_df

,School_Year,District_ID,School_Code,Cooperative_Educational_Service_Agency,District_Name,Overall_Accountability_Score,Overall_Accountability_Rating,Lowest_Grade_in_the_District,Highest_Grade_in_the_District,School_Type,...,Percent_Economically_Disadvantaged,Percent_Limited_English_Proficient,Percent_Within_District_Mobility,Percent_Between_District_Mobility,District_Student_Achievement_Score,District_ELA_Achievement_Score,District_Mathematics_Achievement_Score,District_Student_Growth_Score,District_ELA_Growth_Score,District_Mathematics_Growth_Score
0,2020-21,2450,20,Arrowhead UHS,Arrowhead High,85.2,Significantly Exceeds Expectations,9,12,High School,...,0.084,0.003,0.0,0.011,88.1,84.6,91.6,71.7,58.4,85.0
1,2020-21,6013,20,Big Foot UHS,Big Foot High,71.5,Exceeds Expectations,9,12,High School,...,0.423,0.047,0.0,0.013,60.6,65.7,55.5,71.7,77.4,66.0
2,2020-21,657,20,Brighton #1,Brighton Elementary,83.4,Significantly Exceeds Expectations,K4,8,Elementary School,...,0.17,0.005,0.0,0.025,88.9,86.6,91.2,65.1,60.3,69.8
3,2020-21,665,20,Bristol #1,Bristol Elementary,80.3,Exceeds Expectations,K4,8,Elementary School,...,0.224,0.009,0.0,0.015,75.3,76.3,74.2,75.5,81.2,69.8
4,2020-21,5054,20,Central/Westosha UHS,Central High,71.2,Exceeds Expectations,9,12,High School,...,0.213,0.012,0.0,0.027,71.6,73.8,69.4,70.8,66.0,75.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2018-19,6083,20,Waterford UHS,Waterford High,79.6,Exceeds Expectations^,9,12,High School,...,0.094,0.005,0.0,0.017,74.4,37.2,37.2,78.4,33.0,45.4
1260,2018-19,6412,20,Wheatland J1,Wheatland Center Elementary,71,Meets Expectations,K4,8,Elementary School,...,0.458,0.002,0.0,0.026,74.2,37.3,36.9,68.0,37.8,30.2
1261,2018-19,6545,20,Wilmot UHS,Wilmot High,70,Meets Expectations,9,12,High School,...,0.288,0.007,0.0,0.054,65.3,33.8,31.5,52.7,29.2,23.5
1262,2018-19,6720,20,Woodruff J1,Arbor Vitae-Woodruff Elementary,81.8,Exceeds Expectations,K4,8,Elementary School,...,0.514,0,0.0,0.033,77.4,39.3,38.1,78.4,35.9,42.5


In [9]:
#checking the data types
reportcard_df.dtypes

School_Year                                           object
District_ID                                            int64
School_Code                                           object
Cooperative_Educational_Service_Agency                object
District_Name                                         object
Overall_Accountability_Score                          object
Overall_Accountability_Rating                         object
Lowest_Grade_in_the_District                          object
Highest_Grade_in_the_District                          int64
School_Type                                           object
District_Enrollment                                    int64
Percent_American_Indian_or_Alaskan_Native            float64
Percent_Asian                                        float64
Percent_Black_or_African_American                    float64
Percent_Hispanic_Latino                              float64
Percent_Native_Hawaiian_or_Other_Pacific_Islander    float64
Percent_White           

In [10]:
for col in reportcard_df.columns:
    print(col)

School_Year
District_ID
School_Code
Cooperative_Educational_Service_Agency
District_Name
Overall_Accountability_Score
Overall_Accountability_Rating
Lowest_Grade_in_the_District
Highest_Grade_in_the_District
School_Type
District_Enrollment
Percent_American_Indian_or_Alaskan_Native
Percent_Asian
Percent_Black_or_African_American
Percent_Hispanic_Latino
Percent_Native_Hawaiian_or_Other_Pacific_Islander
Percent_White
Percent_Two_or_More_Races
Percent_Students_with_Disabilities
Percent_Economically_Disadvantaged
Percent_Limited_English_Proficient
Percent_Within_District_Mobility
Percent_Between_District_Mobility
District_Student_Achievement_Score
District_ELA_Achievement_Score
District_Mathematics_Achievement_Score
District_Student_Growth_Score
District_ELA_Growth_Score
District_Mathematics_Growth_Score


In [11]:
# checking the null values but not removing due to data loss
reportcard_df.isnull().sum()

School_Year                                             0
District_ID                                             0
School_Code                                           379
Cooperative_Educational_Service_Agency                  0
District_Name                                           0
Overall_Accountability_Score                            3
Overall_Accountability_Rating                           3
Lowest_Grade_in_the_District                            0
Highest_Grade_in_the_District                           0
School_Type                                          1131
District_Enrollment                                     0
Percent_American_Indian_or_Alaskan_Native               0
Percent_Asian                                           0
Percent_Black_or_African_American                       0
Percent_Hispanic_Latino                                 0
Percent_Native_Hawaiian_or_Other_Pacific_Islander       0
Percent_White                                           0
Percent_Two_or

In [12]:
reportcard_df

,School_Year,District_ID,School_Code,Cooperative_Educational_Service_Agency,District_Name,Overall_Accountability_Score,Overall_Accountability_Rating,Lowest_Grade_in_the_District,Highest_Grade_in_the_District,School_Type,...,Percent_Economically_Disadvantaged,Percent_Limited_English_Proficient,Percent_Within_District_Mobility,Percent_Between_District_Mobility,District_Student_Achievement_Score,District_ELA_Achievement_Score,District_Mathematics_Achievement_Score,District_Student_Growth_Score,District_ELA_Growth_Score,District_Mathematics_Growth_Score
0,2020-21,2450,20,Arrowhead UHS,Arrowhead High,85.2,Significantly Exceeds Expectations,9,12,High School,...,0.084,0.003,0.0,0.011,88.1,84.6,91.6,71.7,58.4,85.0
1,2020-21,6013,20,Big Foot UHS,Big Foot High,71.5,Exceeds Expectations,9,12,High School,...,0.423,0.047,0.0,0.013,60.6,65.7,55.5,71.7,77.4,66.0
2,2020-21,657,20,Brighton #1,Brighton Elementary,83.4,Significantly Exceeds Expectations,K4,8,Elementary School,...,0.17,0.005,0.0,0.025,88.9,86.6,91.2,65.1,60.3,69.8
3,2020-21,665,20,Bristol #1,Bristol Elementary,80.3,Exceeds Expectations,K4,8,Elementary School,...,0.224,0.009,0.0,0.015,75.3,76.3,74.2,75.5,81.2,69.8
4,2020-21,5054,20,Central/Westosha UHS,Central High,71.2,Exceeds Expectations,9,12,High School,...,0.213,0.012,0.0,0.027,71.6,73.8,69.4,70.8,66.0,75.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2018-19,6083,20,Waterford UHS,Waterford High,79.6,Exceeds Expectations^,9,12,High School,...,0.094,0.005,0.0,0.017,74.4,37.2,37.2,78.4,33.0,45.4
1260,2018-19,6412,20,Wheatland J1,Wheatland Center Elementary,71,Meets Expectations,K4,8,Elementary School,...,0.458,0.002,0.0,0.026,74.2,37.3,36.9,68.0,37.8,30.2
1261,2018-19,6545,20,Wilmot UHS,Wilmot High,70,Meets Expectations,9,12,High School,...,0.288,0.007,0.0,0.054,65.3,33.8,31.5,52.7,29.2,23.5
1262,2018-19,6720,20,Woodruff J1,Arbor Vitae-Woodruff Elementary,81.8,Exceeds Expectations,K4,8,Elementary School,...,0.514,0,0.0,0.033,77.4,39.3,38.1,78.4,35.9,42.5


In [13]:
# removing columns 'School_Code','School_Type'
reportcard_df = reportcard_df.drop(columns=['School_Code','School_Type'])


In [15]:
#changing data type to string
reportcard_df['Cooperative_Educational_Service_Agency'] = reportcard_df['Cooperative_Educational_Service_Agency'].astype("string")



In [17]:
#changing data type to string
reportcard_df['Lowest_Grade_in_the_District'] = reportcard_df['Lowest_Grade_in_the_District'].astype("string")


In [18]:
#changing data type to string
reportcard_df['Highest_Grade_in_the_District'] = reportcard_df['Highest_Grade_in_the_District'].astype("string")


In [19]:
#Removing NA and AR values from Percent_Students_with_Disabilities
reportcard_df= reportcard_df.dropna(subset =['Overall_Accountability_Score'])

In [27]:
#Removing NA and AR values from Percent_Students_with_Disabilities
reportcard_df = reportcard_df[reportcard_df['Overall_Accountability_Score']!='AR']

In [21]:
#changing data type to float
reportcard_df['Overall_Accountability_Score'] =reportcard_df['Overall_Accountability_Score'].astype(float)

In [22]:
#changing data type to float
reportcard_df['Percent_Students_with_Disabilities'] =reportcard_df['Percent_Students_with_Disabilities'].astype(float)

In [23]:
#changing data type to float
reportcard_df['Percent_Limited_English_Proficient'] =reportcard_df['Percent_Limited_English_Proficient'].astype(float)

In [25]:
#changing data type to float
reportcard_df['Percent_Economically_Disadvantaged'] =reportcard_df['Percent_Economically_Disadvantaged'].astype(float)

In [30]:
reportcard_df

,School_Year,District_ID,Cooperative_Educational_Service_Agency,District_Name,Overall_Accountability_Score,Overall_Accountability_Rating,Lowest_Grade_in_the_District,Highest_Grade_in_the_District,District_Enrollment,Percent_American_Indian_or_Alaskan_Native,...,Percent_Economically_Disadvantaged,Percent_Limited_English_Proficient,Percent_Within_District_Mobility,Percent_Between_District_Mobility,District_Student_Achievement_Score,District_ELA_Achievement_Score,District_Mathematics_Achievement_Score,District_Student_Growth_Score,District_ELA_Growth_Score,District_Mathematics_Growth_Score
0,2020-21,2450,Arrowhead UHS,Arrowhead High,85.2,Significantly Exceeds Expectations,9,12,2084,0.000,...,0.084,0.003,0.0,0.011,88.1,84.6,91.6,71.7,58.4,85.0
1,2020-21,6013,Big Foot UHS,Big Foot High,71.5,Exceeds Expectations,9,12,468,0.006,...,0.423,0.047,0.0,0.013,60.6,65.7,55.5,71.7,77.4,66.0
2,2020-21,657,Brighton #1,Brighton Elementary,83.4,Significantly Exceeds Expectations,K4,8,200,0.000,...,0.170,0.005,0.0,0.025,88.9,86.6,91.2,65.1,60.3,69.8
3,2020-21,665,Bristol #1,Bristol Elementary,80.3,Exceeds Expectations,K4,8,803,0.000,...,0.224,0.009,0.0,0.015,75.3,76.3,74.2,75.5,81.2,69.8
4,2020-21,5054,Central/Westosha UHS,Central High,71.2,Exceeds Expectations,9,12,1131,0.001,...,0.213,0.012,0.0,0.027,71.6,73.8,69.4,70.8,66.0,75.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2018-19,6083,Waterford UHS,Waterford High,79.6,Exceeds Expectations^,9,12,1058,0.000,...,0.094,0.005,0.0,0.017,74.4,37.2,37.2,78.4,33.0,45.4
1260,2018-19,6412,Wheatland J1,Wheatland Center Elementary,71.0,Meets Expectations,K4,8,542,0.002,...,0.458,0.002,0.0,0.026,74.2,37.3,36.9,68.0,37.8,30.2
1261,2018-19,6545,Wilmot UHS,Wilmot High,70.0,Meets Expectations,9,12,1026,0.002,...,0.288,0.007,0.0,0.054,65.3,33.8,31.5,52.7,29.2,23.5
1262,2018-19,6720,Woodruff J1,Arbor Vitae-Woodruff Elementary,81.8,Exceeds Expectations,K4,8,545,0.077,...,0.514,0.000,0.0,0.033,77.4,39.3,38.1,78.4,35.9,42.5


In [26]:
#New dtypes
reportcard_df.dtypes

School_Year                                           object
District_ID                                            int64
Cooperative_Educational_Service_Agency                string
District_Name                                         object
Overall_Accountability_Score                         float64
Overall_Accountability_Rating                         object
Lowest_Grade_in_the_District                          string
Highest_Grade_in_the_District                         string
District_Enrollment                                    int64
Percent_American_Indian_or_Alaskan_Native            float64
Percent_Asian                                        float64
Percent_Black_or_African_American                    float64
Percent_Hispanic_Latino                              float64
Percent_Native_Hawaiian_or_Other_Pacific_Islander    float64
Percent_White                                        float64
Percent_Two_or_More_Races                            float64
Percent_Students_with_Di